## Draw Bounding Boxes on Images with the Roboflow Python SDK and Supervision
<p>
  <a align="center" href="" target="_blank">
    <img
      width="60%"
      src="https://raw.githubusercontent.com/roboflow/roboflow-computer-vision-utilities/main/figures/roboflow-cv-utilities-header.png"
    >
  </a>
</p>

## Installing Dependencies

In [1]:
!pip install -q roboflow>=1.0.5
!pip install -q supervision>=0.6.0

### Setting up our utility functions

In [8]:
import os
import json
import cv2
import numpy as np
from roboflow import Roboflow

import supervision as sv
from supervision.draw.color import Color
from supervision.draw.color import ColorPalette
from supervision import Detections, BoxAnnotator


def load_roboflow_model(api_key, workspace_id, project_id, version_number):

    # authenticate to your Roboflow account and load your model
    rf = Roboflow(api_key=api_key)
    project = rf.workspace(workspace_id).project(project_id)
    version = project.version(version_number)
    model = version.model
    
    return project, model

Receiving Model Predictions

In [9]:
def make_prediction(project, model, image_path, confidence, overlap):

    # load the image and make predictions with your model
    img = cv2.imread(image_path)
    predictions = model.predict(image_path, confidence=confidence, overlap=overlap)
    predictions_json = predictions.json()
    roboflow_xyxy = np.empty((0, 4))
    predicted_classes = []
    for bounding_box in predictions:
        x1 = bounding_box['x'] - bounding_box['width'] / 2
        x2 = bounding_box['x'] + bounding_box['width'] / 2
        y1 = bounding_box['y'] - bounding_box['height'] / 2
        y2 = bounding_box['y'] + bounding_box['height'] / 2
        np.vstack((roboflow_xyxy, [x1, y1, x2, y2]))
        predicted_classes.append(bounding_box['class'])
        
        # class_name = bounding_box['class']
        # confidence = bounding_box['confidence']
        sv_xyxy = Detections(roboflow_xyxy).from_roboflow(
            predictions_json,class_list=list((project.classes).keys()))

    return img, predictions_json, sv_xyxy, predicted_classes

#### [Supervision](https://github.com/roboflow/supervision/)
A set of easy-to-use utils that will come in handy in any computer vision project. Supervision is still in pre-release stage. 🚧 Keep your eyes open for potential bugs and be aware that at this stage our API is still fluid and may change.
<p>
  <a align="center" href="" target="_blank">
    <img
      width="60%"
      src="https://media.roboflow.com/open-source/supervision/rf-supervision-banner.png?updatedAt=1678995927529"
    >
  </a>
</p>

#### [Roboflow Response Object Format](https://docs.roboflow.com/inference/hosted-api#response-object-format) (Object Detection)
Draw Bounding Boxes on Images with the Roboflow Python SDK and Supervision

<p>
  <a align="center" href="" target="_blank">
    <img
      width="50%"
      src="https://raw.githubusercontent.com/roboflow/roboflow-computer-vision-utilities/main/figures/ResponseObjectFormat_JSON.png"
    >
  </a>
</p>

In [21]:
def fill_bounding_boxes(image, sv_xyxy, class_ids, add_labels):

    #set add_labels to True to show the label for each object
    image_with_boxes = BoxAnnotator(
        color=Color.black(), thickness=-1).annotate(image, sv_xyxy, labels=class_ids, skip_label=add_labels)

    return image_with_boxes

Saving the Image with Bounding Boxes

In [11]:
def save_image(image, original_image_path, output_directory="results"):

    os.makedirs(output_directory, exist_ok=True)
    filename = os.path.basename(original_image_path)
    output_path = os.path.join(output_directory, f"result_{filename}")
    cv2.imwrite(output_path, image)
    return output_path

#### Don't forgot to add your image file(s)!

### Running the whole thing!

In [20]:
def main():

    ## Authentication info to load the model
    ## Sample project: https://universe.roboflow.com/roboflow-universe-projects/construction-site-safety/model/25
    ## Workspace ID: "roboflow-universe-projects", Project ID: "construction-site-safety", Version Number: 25
    api_key = "ROBOFLOW_PRIVATE_API_KEY"
    workspace_id = "ROBOFLOW_WORKSPACE_ID"
    project_id = "ROBOFLOW_POJECT_ID"
    version_number = "ROBOFLOW_VERSION_NUMBER"
    project, model = load_roboflow_model(api_key, workspace_id, project_id, version_number)

    # Make a prediction on the specified image file
    image_path = "/content/test_images/construction-2-_mp4-13_jpg.rf.dcfc8cd7a7c9f6109ff18cab2b50f6bc.jpg"
    confidence = 40
    overlap = 30
    image, predictions_json, pred_sv_xyxy, predicted_classes = make_prediction(
        project, model, image_path, confidence, overlap)

    print(predictions_json)

    ## Set add_labels to False to draw class labels on the bounding boxes
    add_labels = True
    for i in range(len(pred_sv_xyxy)):
        image_with_boxes = fill_bounding_boxes(image, pred_sv_xyxy, predicted_classes, add_labels)

    # Save the image with bounding boxes for the detected objects drawn on them
    output_path = save_image(image_with_boxes, image_path)

    print(f"The image has been processed and saved to {output_path}")

if __name__ == "__main__":
    main()


loading Roboflow workspace...
loading Roboflow project...
{'predictions': [{'x': 1113, 'y': 363, 'width': 334, 'height': 280, 'confidence': 0.9350578784942627, 'class': 'machinery', 'image_path': '/content/construction-2-_mp4-162_jpg.rf.edc08e7528429b4315f59637aaf65cf0.jpg', 'prediction_type': 'ObjectDetectionModel'}, {'x': 307, 'y': 314, 'width': 582, 'height': 460, 'confidence': 0.8053478002548218, 'class': 'machinery', 'image_path': '/content/construction-2-_mp4-162_jpg.rf.edc08e7528429b4315f59637aaf65cf0.jpg', 'prediction_type': 'ObjectDetectionModel'}, {'x': 905, 'y': 272, 'width': 722, 'height': 481, 'confidence': 0.7803330421447754, 'class': 'machinery', 'image_path': '/content/construction-2-_mp4-162_jpg.rf.edc08e7528429b4315f59637aaf65cf0.jpg', 'prediction_type': 'ObjectDetectionModel'}], 'image': {'width': '1280', 'height': '720'}}
The image has been processed and saved to results/result_construction-2-_mp4-162_jpg.rf.edc08e7528429b4315f59637aaf65cf0.jpg
